In [2]:
import yfinance as yf
import pandas as pd
from datetime import datetime
import time

In [3]:
symbol = 'gme'

In [5]:
# Fetch the option data
stock= yf.Ticker(symbol)

# create an empty list
calls_bucket = []
puts_bucket = []

# iterate through dataframes for each expiration date
for i in range(0, len(stock.options)):
  option = stock.option_chain(stock.options[i])

# append each dataframe to the list
  calls_bucket.append(option.calls)
  puts_bucket.append(option.puts)

In [6]:
# concatenate dataframes 
df_calls = pd.concat(calls_bucket)
df_puts = pd.concat(puts_bucket)
df_calls.head()

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,GME230217C00003000,2023-01-23 18:01:29+00:00,3.00,19.38,15.50,17.15,0.0,0.0,NaN,0.0,14.343751,True,REGULAR,USD
1,GME230217C00006250,2022-12-28 18:32:24+00:00,6.25,11.25,16.05,17.00,0.0,0.0,3.0,0.0,21.562503,True,REGULAR,USD
2,GME230217C00007500,2023-01-12 18:37:54+00:00,7.50,11.85,11.40,12.10,0.0,0.0,1.0,0.0,0.000010,True,REGULAR,USD
3,GME230217C00008750,2022-11-15 17:09:51+00:00,8.75,19.00,13.45,14.75,0.0,0.0,1.0,0.0,16.015630,True,REGULAR,USD
4,GME230217C00010000,2023-02-07 15:40:39+00:00,10.00,11.21,8.90,10.10,0.0,0.0,5.0,37.0,5.312503,True,REGULAR,USD


In [7]:
# Module to add deltaOI and deltaPrice

In [8]:
# Module to add StockPrice
def get_current_price(symbol):
    ticker = yf.Ticker(symbol)
    todays_data = ticker.history(period='1d')
    return todays_data['Close'][0]

# call the function to get the stock price
current_price = get_current_price(symbol)

# add the stock price to the dataframe
df_calls['StockPrice'] = current_price
df_puts['StockPrice'] = current_price
df_calls.head()

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,StockPrice
0,GME230217C00003000,2023-01-23 18:01:29+00:00,3.00,19.38,15.50,17.15,0.0,0.0,NaN,0.0,14.343751,True,REGULAR,USD,19.870001
1,GME230217C00006250,2022-12-28 18:32:24+00:00,6.25,11.25,16.05,17.00,0.0,0.0,3.0,0.0,21.562503,True,REGULAR,USD,19.870001
2,GME230217C00007500,2023-01-12 18:37:54+00:00,7.50,11.85,11.40,12.10,0.0,0.0,1.0,0.0,0.000010,True,REGULAR,USD,19.870001
3,GME230217C00008750,2022-11-15 17:09:51+00:00,8.75,19.00,13.45,14.75,0.0,0.0,1.0,0.0,16.015630,True,REGULAR,USD,19.870001
4,GME230217C00010000,2023-02-07 15:40:39+00:00,10.00,11.21,8.90,10.10,0.0,0.0,5.0,37.0,5.312503,True,REGULAR,USD,19.870001


In [9]:
# Module to filter data

def filter_options_data(df_calls, df_puts, filter=False):
    # Calculate the call upside % threshold for strike
    cPS = 0.03
    cstrike_threshold = df_calls['StockPrice'] + (df_calls['StockPrice'] * cPS)

    # Calculate the call % premium to stock price threshold
    cPP = 0.05
    clastPrice_threshold = df_calls['StockPrice'] * cPP

    # Calculate the put downside % threshold for strike
    pPS = 0.05
    pstrike_threshold = df_puts['StockPrice'] - (df_puts['StockPrice'] * pPS)

    # Calculate the put % premium to stock price threshold
    pPP = 0.03
    plastPrice_threshold = df_puts['StockPrice'] * pPP

    if filter:
        # Filter the Calls data based on the strike and premium criteria
        df_calls = df_calls[(df_calls['strike'] >= cstrike_threshold) & (df_calls['lastPrice'] >= clastPrice_threshold)]

        # Filter the Puts data based on the strike and premium criteria
        df_puts = df_puts[(df_puts['strike'] >= pstrike_threshold) & (df_puts['lastPrice'] >= plastPrice_threshold)]

    # Reset the index of the cleaned dataframe
    df_calls = df_calls.reset_index(drop=True)
    df_puts = df_puts.reset_index(drop=True)

    return df_calls, df_puts

In [10]:
# set to true/false
filter_options_data(df_calls, df_puts, filter=False)
df_calls.head()

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,StockPrice
0,GME230217C00003000,2023-01-23 18:01:29+00:00,3.00,19.38,15.50,17.15,0.0,0.0,NaN,0.0,14.343751,True,REGULAR,USD,19.870001
1,GME230217C00006250,2022-12-28 18:32:24+00:00,6.25,11.25,16.05,17.00,0.0,0.0,3.0,0.0,21.562503,True,REGULAR,USD,19.870001
2,GME230217C00007500,2023-01-12 18:37:54+00:00,7.50,11.85,11.40,12.10,0.0,0.0,1.0,0.0,0.000010,True,REGULAR,USD,19.870001
3,GME230217C00008750,2022-11-15 17:09:51+00:00,8.75,19.00,13.45,14.75,0.0,0.0,1.0,0.0,16.015630,True,REGULAR,USD,19.870001
4,GME230217C00010000,2023-02-07 15:40:39+00:00,10.00,11.21,8.90,10.10,0.0,0.0,5.0,37.0,5.312503,True,REGULAR,USD,19.870001


In [11]:
# Concatenate dataframes 
df_calls_puts = pd.concat([df_calls, df_puts], axis=0)
df_calls_puts.reset_index(drop=True, inplace=True)
df_calls_puts.head()

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,StockPrice
0,GME230217C00003000,2023-01-23 18:01:29+00:00,3.00,19.38,15.50,17.15,0.0,0.0,NaN,0.0,14.343751,True,REGULAR,USD,19.870001
1,GME230217C00006250,2022-12-28 18:32:24+00:00,6.25,11.25,16.05,17.00,0.0,0.0,3.0,0.0,21.562503,True,REGULAR,USD,19.870001
2,GME230217C00007500,2023-01-12 18:37:54+00:00,7.50,11.85,11.40,12.10,0.0,0.0,1.0,0.0,0.000010,True,REGULAR,USD,19.870001
3,GME230217C00008750,2022-11-15 17:09:51+00:00,8.75,19.00,13.45,14.75,0.0,0.0,1.0,0.0,16.015630,True,REGULAR,USD,19.870001
4,GME230217C00010000,2023-02-07 15:40:39+00:00,10.00,11.21,8.90,10.10,0.0,0.0,5.0,37.0,5.312503,True,REGULAR,USD,19.870001


In [12]:
# Function to keep only the required columns and add new columns
def process_data(df, drop_col=False, add_col=False):
    if drop_col:
        df = df.loc[:, ['contractSymbol','strike', 'lastPrice', 'openInterest', 'StockPrice']]
    if add_col:
        df.loc[:, 'premiumPrice'] = (df['lastPrice'] / df['StockPrice'])*100
        df.loc[:, 'strike_diff'] = ((df['strike'] - df['StockPrice'])/df['StockPrice'])*100
    return df

In [13]:
# Set to True/False to activate filtering
df_processed = process_data(df_calls, drop_col=True, add_col=True)
df_processed.head()

,contractSymbol,strike,lastPrice,openInterest,StockPrice,premiumPrice,strike_diff
0,GME230217C00003000,3.00,19.38,0.0,19.870001,97.533967,-84.901863
1,GME230217C00006250,6.25,11.25,0.0,19.870001,56.618015,-68.545547
2,GME230217C00007500,7.50,11.85,0.0,19.870001,59.637642,-62.254657
3,GME230217C00008750,8.75,19.00,0.0,19.870001,95.621536,-55.963766
4,GME230217C00010000,10.00,11.21,37.0,19.870001,56.416706,-49.672876


In [15]:
def get_processed_dataframe():
    return df_processed